In [16]:
import pandas as pd


In [17]:
df = pd.read_csv('../data/output/unlabeled_withBPAnnotations.csv')
df.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName
0,2a584093-fb6c-4a22-9c31-ac835d2b474a.xml,828,846,PREF,ATRIAL FIBRILLATION,Digoxin is a cardiac glycoside indicated for:|...,http://purl.obolibrary.org/obo/DOID_0060224,DB00390,DIGOXIN
1,5b0297ba-497e-46af-b027-6d46d9409384.xml,902,920,PREF,ATRIAL FIBRILLATION,"Digoxin Tablets, USP are a cardiac glycoside i...",http://purl.obolibrary.org/obo/DOID_0060224,DB00390,DIGOXIN
2,83793809-2e55-366b-e053-2991aa0a414f.xml,412,430,PREF,ATRIAL FIBRILLATION,DIGOXIN is a cardiac glycoside indicated for:|...,http://purl.obolibrary.org/obo/DOID_0060224,DB00390,DIGOXIN
3,83793809-2e55-366b-e053-2991aa0a414f.xml,1101,1119,PREF,ATRIAL FIBRILLATION,DIGOXIN is a cardiac glycoside indicated for:|...,http://purl.obolibrary.org/obo/DOID_0060224,DB00390,DIGOXIN
4,83d37d5a-ed63-e142-e053-2991aa0ad846.xml,412,430,PREF,ATRIAL FIBRILLATION,DIGOXIN is a cardiac glycoside indicated for:|...,http://purl.obolibrary.org/obo/DOID_0060224,DB00390,DIGOXIN


In [18]:
len(df)

25670

In [19]:
# Download and process UMLS to DOID mappings
# We use the propagated mappings here
url = 'https://raw.githubusercontent.com/dhimmel/disease-ontology/72614ade9f1cc5a5317b8f6836e1e464b31d5587/data/xrefs-prop-slim.tsv'
domap_df = pd.read_table(url)
domap_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


,doid_code,doid_name,resource,resource_id
0,DOID:2531,hematologic cancer,CSP,2004-1600
1,DOID:2531,hematologic cancer,CSP,2004-1803
2,DOID:2531,hematologic cancer,CSP,2004-2820
3,DOID:2531,hematologic cancer,EFO,0000095
4,DOID:2531,hematologic cancer,EFO,0000096


In [20]:
domap_df= domap_df[domap_df['resource'] == "UMLS"]


In [21]:
domap_df['diseaseId'] = domap_df['resource_id']
domap_df = domap_df.drop(columns=['resource'])
domap_df.head()

,doid_code,doid_name,resource_id,diseaseId
513,DOID:2531,hematologic cancer,C0001815,C0001815
514,DOID:2531,hematologic cancer,C0002890,C0002890
515,DOID:2531,hematologic cancer,C0006413,C0006413
516,DOID:2531,hematologic cancer,C0019829,C0019829
517,DOID:2531,hematologic cancer,C0023418,C0023418


In [22]:
def trimName(url):
    return  url.replace("http://purl.obolibrary.org/obo/","").replace("_",':')

df.DO_ID= df.DO_ID.apply(trimName)

In [23]:
domap_df = domap_df[['doid_code', 'diseaseId']]   
domap_df.rename(columns={'doid_code':'DO_ID','diseaseId':'disease'}, inplace=True)

domap_df.head()


,DO_ID,disease
513,DOID:2531,C0001815
514,DOID:2531,C0002890
515,DOID:2531,C0006413
516,DOID:2531,C0019829
517,DOID:2531,C0023418


In [24]:
#Merge the DOID mapping to the BPAnnotator
df2 = df.merge(domap_df, on=["DO_ID"], how='inner')
df2.head()

,Label_ID,From,To,Type,Annotation,Context,DO_ID,DB_ID,DrugName,disease
0,00dd39bc-874e-4250-935d-99882ca0e558.xml,138,143,PREF,ASTHMA,Montelukast sodium is a leukotriene receptor a...,DOID:2841,DB00471,MONTELUKAST,C0004096
1,00dd39bc-874e-4250-935d-99882ca0e558.xml,138,143,PREF,ASTHMA,Montelukast sodium is a leukotriene receptor a...,DOID:2841,DB00471,MONTELUKAST,C0038218
2,00dd39bc-874e-4250-935d-99882ca0e558.xml,138,143,PREF,ASTHMA,Montelukast sodium is a leukotriene receptor a...,DOID:2841,DB00471,MONTELUKAST,C0155877
3,00dd39bc-874e-4250-935d-99882ca0e558.xml,138,143,PREF,ASTHMA,Montelukast sodium is a leukotriene receptor a...,DOID:2841,DB00471,MONTELUKAST,C0155880
4,00dd39bc-874e-4250-935d-99882ca0e558.xml,138,143,PREF,ASTHMA,Montelukast sodium is a leukotriene receptor a...,DOID:2841,DB00471,MONTELUKAST,C0694548


In [25]:
df2.rename(columns={'DB_ID':'Drug','disease':'Disease'},inplace=True)

In [27]:
df2.to_csv("../data/output/final_unlabeled.csv", index=False)